In [ ]:
import osmnx as ox
places = ["Praha, Czech Republic", "Středočeský kraj, Czech Republic"]
graph = ox.graph_from_place(places, network_type="drive")
graph = graph.to_undirected()
ox.save_graphml(graph, filepath='praha_stredocesky.graphml')

In [ ]:
import osmnx as ox
import networkx as nx
import json
import random
import numpy as np
from datetime import datetime, timedelta
import geopy.distance
import math

places = ["Praha, Czech Republic", "Středočeský kraj, Czech Republic"]
graph = ox.graph_from_place(places, network_type="drive")
graph = graph.to_undirected()
ox.save_graphml(graph, filepath='praha_stredocesky.graphml')

# Načítavame uložený graf zo súboru
print("🔄 Načítavam uložený graf...")
graph = ox.load_graphml("praha_stredocesky.graphml")
graph = graph.to_undirected()
print(f"✅ Graf načítaný – {len(graph.nodes)} uzlov, {len(graph.edges)} hrán")

def generate_weather():
    weather = random.choices(['clear', 'rain', 'fog', 'snow'], weights=[0.5, 0.2, 0.2, 0.1])[0]
    temp = random.uniform(-5, 30) if weather != 'snow' else random.uniform(-10, 0)
    visibility = random.uniform(100, 1000) if weather in ['fog', 'snow'] else random.uniform(5000, 10000)
    if weather == 'rain' and random.random() < 0.3:
        visibility = random.uniform(100, 500)
    wind = random.uniform(0, 15)
    road_state = random.choice(['dry', 'wet', 'icy'])
    return {"condition": weather, "temperature": round(temp, 1), "visibility": round(visibility), "wind": round(wind, 1), "road_state": road_state}

def generate_traffic(highway_type):
    if highway_type in ['motorway', 'trunk']:
        return random.choice(['low', 'medium'])
    elif highway_type in ['primary', 'secondary']:
        return random.choice(['medium', 'high'])
    else:
        return random.choice(['low', 'medium', 'high', 'very_high'])

def estimate_speed_limit(highway_type):
    speed_limits = {
        'motorway': 130,
        'trunk': 110,
        'primary': 90,
        'secondary': 70,
        'tertiary': 50,
        'residential': 30,
        'service': 20
    }
    if isinstance(highway_type, list):
        highway_type = highway_type[0]
    return speed_limits.get(highway_type, 50)

def estimate_curve_speed(lat1, lon1, lat2, lon2, lat3, lon3):
    try:
        a = geopy.distance.distance((lat2, lon2), (lat3, lon3)).m
        b = geopy.distance.distance((lat1, lon1), (lat3, lon3)).m
        c = geopy.distance.distance((lat1, lon1), (lat2, lon2)).m
        s = (a + b + c) / 2
        area = math.sqrt(s * (s - a) * (s - b) * (s - c))
        r = (a * b * c) / (4 * area)
        if r < 50:
            return min(50, math.sqrt(0.35 * 9.81 * r) * 3.6)
        return 130
    except:
        return 130

def get_osmnx_path_and_coords(graph, scenario, max_attempts=50):
    nodes = list(graph.nodes)
    for _ in range(max_attempts):
        start, end = random.sample(nodes, 2)
        try:
            path = nx.shortest_path(graph, source=start, target=end, weight='length')
            coords, highway_types = [], []
            for u, v in zip(path[:-1], path[1:]):
                edge_data = graph.get_edge_data(u, v)
                edge = edge_data[0] if edge_data else {}
                coords += [(graph.nodes[u]['y'], graph.nodes[u]['x']), (graph.nodes[v]['y'], graph.nodes[v]['x'])]
                highway_types.append(edge.get('highway', 'unknown'))
            if scenario == 'mesto' and all(ht not in ['motorway', 'trunk'] for ht in highway_types):
                return path, coords, highway_types
            elif scenario == 'diaľnica' and any(ht in ['motorway', 'trunk'] for ht in highway_types):
                return path, coords, highway_types
            elif scenario == 'vidiek' and any(ht in ['primary', 'secondary'] for ht in highway_types):
                return path, coords, highway_types
        except:
            continue
    raise ValueError("❌ Nepodarilo sa nájsť vhodnú trasu.")

def compute_speed_and_acceleration(coords, highway_types, driver_type, weather, traffic):
    speeds, accelerations = [], []
    for i in range(len(coords)):
        highway = highway_types[min(i, len(highway_types)-1)]
        base = estimate_speed_limit(highway)
        tf = {"low": 1.0, "medium": 0.8, "high": 0.6, "very_high": 0.4}[traffic]
        wf = 0.7 if weather in ['rain','snow','fog'] else 1.0
        df = {"defenzívny": 0.75, "normálny": 1.0, "agresívny": 1.25}[driver_type]
        speed = base * tf * wf * df
        if 1 < i < len(coords)-1:
            curve_speed = estimate_curve_speed(*coords[i-1], *coords[i], *coords[i+1])
            speed = min(speed, curve_speed)
        if highway in ['residential', 'tertiary', 'service'] and random.random() < 0.05:
            speed = random.uniform(0, 10)  # stop&go
        speeds.append(max(random.gauss(speed, 3), 5))
    speeds = [speeds[0]] + speeds
    for i in range(1, len(speeds)):
        delta_v = speeds[i] - speeds[i-1]
        if random.random() < 0.02:
            accel = random.uniform(-7.0, -5.0)  # emergency braking
        else:
            accel = delta_v / 2
        accel = max(min(accel, 5.0), -7.0)
        accelerations.append(accel)
    accelerations = [0] + accelerations
    return speeds[1:], accelerations

def generate_incidents(coords, weather, driver_type):
    incidents = []
    for i in range(5, len(coords)-5):
        prob = 0.01
        if weather in ['fog','snow','rain']:
            prob += 0.02
        if driver_type == 'defenzívny':
            prob -= 0.007
        elif driver_type == 'normálny':
            prob += 0.01
        elif driver_type == 'agresívny':
            prob += 0.035
        if random.random() < prob:
            incidents.append(i)
    return incidents

def generate_single_drive(pid, cid, sid, vin, start_time, driver_type, scenario):
    path_nodes, coords, highway_types = get_osmnx_path_and_coords(graph, scenario)
    weather_data = generate_weather()
    traffic = generate_traffic(highway_types[0])
    timestamp_base = int(start_time.timestamp())
    records = [{"v": "1", "t": timestamp_base, "n": "IGNITION_ON"}]
    speeds, accels = compute_speed_and_acceleration(coords, highway_types, driver_type, weather_data['condition'], traffic)
    incidents = generate_incidents(coords, weather_data['condition'], driver_type)
    curr_time = timestamp_base

    for i, (lat, lon) in enumerate(coords):
        lat += random.uniform(-0.00002, 0.00002)
        lon += random.uniform(-0.00002, 0.00002)
        records.append({"v": f"{round(lat, 6)},{round(lon, 6)}", "t": curr_time, "n": "PERSONALIZED_LOCATION"})
        records.append({"v": str(round(speeds[i], 1)), "t": curr_time, "n": "SPEED"})
        records.append({"v": str(round(accels[i], 2)), "t": curr_time, "n": "LONGITUDINAL_ACCELERATION"})
        fuel = round(90 - (i/len(coords))*random.uniform(5,20), 1)
        soc = round(100 - (i/len(coords))*random.uniform(2,10), 1)
        lateral = round(random.uniform(-2, 2), 2)
        if driver_type == 'agresívny' and random.random() < 0.05:
            lateral += random.uniform(-5, 5)
        elif driver_type == 'defenzívny' and random.random() < 0.05:
            lateral += random.uniform(-1.5, 1.5)
        if random.random() < 0.02:
            lateral += random.uniform(-4, 4)  # prudký bočný manéver
        heading = random.randint(0,359)
        incident = 1 if i in incidents else 0
        records.append({"v": str(fuel), "t": curr_time, "n": "FUEL_LVL"})
        records.append({"v": str(soc), "t": curr_time, "n": "STATE_OF_CHARGE"})
        records.append({"v": str(lateral), "t": curr_time, "n": "LATERAL_ACCELERATION"})
        records.append({"v": str(heading), "t": curr_time, "n": "HEADING"})
        records.append({"v": str(incident), "t": curr_time, "n": "INCIDENT"})

        if i > 0:
            distance_m = geopy.distance.distance(coords[i-1], coords[i]).meters
            expected_time = distance_m / (speeds[i] / 3.6)
            noise_factor = random.uniform(0.9, 1.1)
            delta_seconds = int(expected_time * noise_factor)
            delta_seconds = max(1, min(delta_seconds, 30))
        else:
            delta_seconds = 1
        curr_time += delta_seconds

    records.append({"v": "0", "t": curr_time, "n": "IGNITION_OFF"})
    
    return {
        "partnerId": pid, "campaignId": cid, "ssoId": sid, "vin": vin,
        "timestamp": timestamp_base, "driverType": driver_type, "scenario": scenario,
        "weather": weather_data, "traffic": traffic,
        "records": records, "path_nodes": path_nodes
    }

# Funkcia na generovanie viacerých jázd naraz
def generate_multiple_drives(num_drives, output_file):
    pids = [100 + i for i in range(num_drives)]
    cids = [200 + i for i in range(num_drives)]
    vins = [f"VINCODE{i:04d}" for i in range(num_drives)]
    driver_types = ['defenzívny', 'normálny', 'agresívny']
    scenarios = ['mesto', 'vidiek', 'diaľnica']

    start_time = datetime(2025, 5, 1)

    with open(output_file, 'w') as f:
        for i in range(num_drives):
            pid = pids[i]
            cid = cids[i]
            sid = f"sso_{i}"
            vin = vins[i]
            driver_type = driver_types[i % len(driver_types)]
            scenario = scenarios[(i // len(driver_types)) % len(scenarios)]

            ride_time = start_time + timedelta(minutes=i * 30)

            try:
                ride = generate_single_drive(
                    pid=pid, 
                    cid=cid, 
                    sid=sid, 
                    vin=vin, 
                    start_time=ride_time, 
                    driver_type=driver_type, 
                    scenario=scenario
                )
                f.write(json.dumps(ride) + '\n')
                print(f"✅ Vytvorená jazda {i+1}/{num_drives} — {scenario} ({driver_type})")
            except Exception as e:
                print(f"❌ Nepodarilo sa vygenerovať jazdu {i+1}: {e}")

# Príklad použitia
generate_multiple_drives(900, "synteticke_json_jazdy_final.jsonl")



🔄 Načítavam uložený graf...
✅ Graf načítaný – 95349 uzlov, 126033 hrán
✅ Vytvorená jazda 1/900 — mesto (defenzívny)
✅ Vytvorená jazda 2/900 — mesto (normálny)
✅ Vytvorená jazda 3/900 — mesto (agresívny)
✅ Vytvorená jazda 4/900 — vidiek (defenzívny)
✅ Vytvorená jazda 5/900 — vidiek (normálny)
✅ Vytvorená jazda 6/900 — vidiek (agresívny)
✅ Vytvorená jazda 7/900 — diaľnica (defenzívny)
✅ Vytvorená jazda 8/900 — diaľnica (normálny)
✅ Vytvorená jazda 9/900 — diaľnica (agresívny)
✅ Vytvorená jazda 10/900 — mesto (defenzívny)
✅ Vytvorená jazda 11/900 — mesto (normálny)
✅ Vytvorená jazda 12/900 — mesto (agresívny)
✅ Vytvorená jazda 13/900 — vidiek (defenzívny)
✅ Vytvorená jazda 14/900 — vidiek (normálny)
✅ Vytvorená jazda 15/900 — vidiek (agresívny)
✅ Vytvorená jazda 16/900 — diaľnica (defenzívny)
✅ Vytvorená jazda 17/900 — diaľnica (normálny)
✅ Vytvorená jazda 18/900 — diaľnica (agresívny)
✅ Vytvorená jazda 19/900 — mesto (defenzívny)
✅ Vytvorená jazda 20/900 — mesto (normálny)
✅ Vytvorená ja

In [31]:
import json
import requests
from datetime import datetime
import numpy as np
import time

# Bezpečné zaokrúhľovanie
def safe_round(value, decimals=1):
    if value is None:
        return None
    return round(value, decimals)

# Spoľahlivejšie získavanie počasia s opakovaniami
def get_real_weather(lat, lon, timestamp, retries=3, delay=1.0):
    dt = datetime.utcfromtimestamp(timestamp).strftime("%Y-%m-%dT%H:%M")
    date_str = dt[:10]
    hour_str = dt[11:13]
    url = (
        f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}"
        f"&hourly=temperature_2m,precipitation,cloudcover,visibility,windspeed_10m"
        f"&start_date={date_str}&end_date={date_str}&timezone=UTC"
    )

    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()

            if 'hourly' not in data or len(data['hourly']['temperature_2m']) == 0:
                print("⚠️ API nevrátilo dáta pre tento dátum")
                return None

            index_hour = int(hour_str)
            hourly = data['hourly']
            weather_data = {
                "temperature": hourly['temperature_2m'][index_hour] if index_hour < len(hourly['temperature_2m']) else None,
                "precipitation": hourly['precipitation'][index_hour] if index_hour < len(hourly['precipitation']) else None,
                "cloudcover": hourly['cloudcover'][index_hour] if index_hour < len(hourly['cloudcover']) else None,
                "visibility": hourly['visibility'][index_hour] if index_hour < len(hourly['visibility']) else None,
                "windspeed": hourly['windspeed_10m'][index_hour] if index_hour < len(hourly['windspeed_10m']) else None
            }
            return weather_data

        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay)
                continue
            print("⚠️ Chyba pri získavaní počasia:", e)
            return None

# Výpočet mediánu súradníc
def extract_median_location(records):
    locations = []
    for rec in records:
        if rec['n'] == 'PERSONALIZED_LOCATION':
            lat, lon = map(float, rec['v'].split(','))
            locations.append((lat, lon))
    if locations:
        arr = np.array(locations)
        median_lat, median_lon = np.median(arr, axis=0)
        return median_lat, median_lon
    return None, None

# Hlavný enrichment cyklus
input_file = "synteticke_json_jazdy_final.jsonl"
output_file = "synteticke_json_jazdy_enriched.jsonl"

with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for idx, line in enumerate(fin):
        drive = json.loads(line)
        records = drive['records']
        lat, lon = extract_median_location(records)
        timestamp = drive['timestamp']

        if lat is not None and lon is not None:
            real_weather = get_real_weather(lat, lon, timestamp)

            if real_weather:
                drive['weather']['temperature'] = safe_round(real_weather.get('temperature'), 1)
                drive['weather']['precipitation'] = safe_round(real_weather.get('precipitation'), 1)
                drive['weather']['cloudcover'] = safe_round(real_weather.get('cloudcover'), 1)
                drive['weather']['visibility'] = real_weather.get('visibility')
                drive['weather']['windspeed'] = safe_round(real_weather.get('windspeed'), 1)
            else:
                print(f"⚠️ Preskakujem enrichment pre jazdu {idx+1} (bez dát)")

        fout.write(json.dumps(drive) + "\n")
        time.sleep(1.0)  # väčšia pauza kvôli API limitu

print("✅ Obohatenie dát o reálne počasie úspešne dokončené.")


⚠️ Chyba pri získavaní počasia: ('Connection aborted.', ConnectionResetError(10054, 'Stávající připojení bylo vynuceně ukončeno vzdáleným hostitelem', None, 10054, None))
⚠️ Preskakujem enrichment pre jazdu 603 (bez dát)
⚠️ Chyba pri získavaní počasia: ('Connection aborted.', ConnectionResetError(10054, 'Stávající připojení bylo vynuceně ukončeno vzdáleným hostitelem', None, 10054, None))
⚠️ Preskakujem enrichment pre jazdu 660 (bez dát)
✅ Obohatenie dát o reálne počasie úspešne dokončené.


In [32]:
import json
from datetime import datetime

def enrich_traffic(drive):
    dt = datetime.utcfromtimestamp(drive['timestamp'])
    hour = dt.hour
    scenario = drive.get('scenario', '')

    if scenario == 'diaľnica':
        traffic = 'medium' if 10 <= hour <= 16 else 'low'
    elif scenario == 'mesto':
        if 7 <= hour <= 9 or 15 <= hour <= 18:
            traffic = 'very_high'
        elif 6 <= hour < 7 or 18 < hour <= 20:
            traffic = 'high'
        else:
            traffic = 'medium'
    elif scenario == 'vidiek':
        traffic = 'low' if hour < 6 or hour > 20 else 'medium'
    else:
        traffic = 'medium'

    drive['traffic_api'] = traffic
    return drive

def enrich_road_risk(drive):
    risk_score = 0
    for rec in drive['records']:
        if rec['n'] == 'SPEED':
            speed = float(rec['v'])
            if speed > 90:
                risk_score += 2
            elif speed > 60:
                risk_score += 1
    drive['road_risk_index'] = min(risk_score, 100)
    return drive

def enrich_incident_probability(drive):
    prob = 0.01
    scenario = drive.get('scenario')
    weather = drive.get('weather', {}).get('condition', '')

    if scenario == 'mesto':
        prob += 0.02
    elif scenario == 'vidiek':
        prob += 0.015

    if weather in ['rain', 'fog', 'snow']:
        prob += 0.03

    drive['incident_base_probability'] = round(prob, 3)
    return drive

def enrich_stop_and_go(drive):
    count = sum(1 for rec in drive['records'] if rec['n'] == 'SPEED' and float(rec['v']) < 5)
    drive['stop_and_go_events'] = count
    return drive

def enrich_hard_accelerations(drive):
    count = sum(1 for rec in drive['records'] if rec['n'] == 'LONGITUDINAL_ACCELERATION' and float(rec['v']) > 3)
    drive['hard_acceleration_events'] = count
    return drive

def enrich_hard_braking(drive):
    count = sum(1 for rec in drive['records'] if rec['n'] == 'LONGITUDINAL_ACCELERATION' and float(rec['v']) < -3)
    drive['hard_braking_events'] = count
    return drive

def enrich_lateral_risk(drive):
    count = sum(1 for rec in drive['records'] if rec['n'] == 'LATERAL_ACCELERATION' and abs(float(rec['v'])) > 2)
    drive['lateral_risk_events'] = count
    return drive

def enrich_driver_behavior(drive):
    score = 0
    score += drive.get('hard_acceleration_events', 0) * 1.5
    score += drive.get('hard_braking_events', 0) * 1.2
    score += drive.get('lateral_risk_events', 0) * 1.0
    score += drive.get('stop_and_go_events', 0) * 0.5
    drive['driver_behavior_score'] = round(score, 1)

    if score < 10:
        label = 'low'
    elif score < 30:
        label = 'medium'
    else:
        label = 'high'
    drive['drive_risk_label'] = label
    return drive

# Main pipeline
input_file = "synteticke_json_jazdy_enriched.jsonl"
output_file = "synteticke_json_jazdy_enriched_full.jsonl"

with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for line in fin:
        drive = json.loads(line)

        drive = enrich_traffic(drive)
        drive = enrich_road_risk(drive)
        drive = enrich_incident_probability(drive)
        drive = enrich_stop_and_go(drive)
        drive = enrich_hard_accelerations(drive)
        drive = enrich_hard_braking(drive)
        drive = enrich_lateral_risk(drive)
        drive = enrich_driver_behavior(drive)

        fout.write(json.dumps(drive) + "\n")

print("✅ Enrichment pipeline úspešne dokončený.")


✅ Enrichment pipeline úspešne dokončený.


In [33]:
print("Traffic API:", drive.get('traffic_api'))
print("Road Risk Index:", drive.get('road_risk_index'))
print("Incident Base Probability:", drive.get('incident_base_probability'))
print("Stop and Go Events:", drive.get('stop_and_go_events'))
print("Hard Accelerations:", drive.get('hard_acceleration_events'))
print("Hard Brakes:", drive.get('hard_braking_events'))
print("Lateral Risk Events:", drive.get('lateral_risk_events'))

Traffic API: medium
Road Risk Index: 53
Incident Base Probability: 0.01
Stop and Go Events: 0
Hard Accelerations: 50
Hard Brakes: 52
Lateral Risk Events: 20


In [34]:
import json
import numpy as np
import osmnx as ox
from shapely.geometry import Point

osm_cache = {}

def extract_median_location(records):
    locations = []
    for rec in records:
        if rec['n'] == 'PERSONALIZED_LOCATION':
            try:
                lat, lon = map(float, rec['v'].split(','))
                locations.append((lat, lon))
            except:
                continue
    if locations:
        arr = np.array(locations)
        median_lat, median_lon = np.median(arr, axis=0)
        return median_lat, median_lon
    return None, None

def sanitize_osm_value(value):
    if isinstance(value, list):
        return value[0]
    if isinstance(value, str) and ';' in value:
        return value.split(';')[0]
    return value

def get_osm_features(lat, lon):
    key = (round(lat, 5), round(lon, 5))
    if key in osm_cache:
        return osm_cache[key]

    try:
        point = (lat, lon)
        tags = {"highway": True}
        gdf = ox.features_from_point(point, tags=tags, dist=50)

        if gdf.empty:
            result = {"osm_enriched": False}
        else:
            nearest = gdf.geometry.distance(Point(lon, lat)).idxmin()
            row = gdf.loc[nearest]
            result = {
                "osm_highway": sanitize_osm_value(row.get('highway', 'unknown')),
                "osm_maxspeed": sanitize_osm_value(row.get('maxspeed', 'unknown')),
                "osm_lanes": sanitize_osm_value(row.get('lanes', 'unknown')),
                "osm_surface": sanitize_osm_value(row.get('surface', 'unknown')),
                "osm_junction": sanitize_osm_value(row.get('junction', 'none')),
                "osm_oneway": sanitize_osm_value(row.get('oneway', 'unknown')),
                "osm_enriched": True
            }

    except Exception as e:
        result = {"osm_enriched": False, "osm_error": str(e)}

    osm_cache[key] = result
    return result

# Spracovanie datasetu
input_file = "synteticke_json_jazdy_enriched_full.jsonl"
output_file = "synteticke_json_jazdy_enriched_full_osm_v2.jsonl"

with open(input_file, "r") as fin, open(output_file, "w") as fout:
    for line in fin:
        drive = json.loads(line)
        lat, lon = extract_median_location(drive['records'])

        if lat is not None and lon is not None:
            osm_features = get_osm_features(lat, lon)
            drive.update(osm_features)

        fout.write(json.dumps(drive) + "\n")

print("✅ Vylepšený OSM enrichment hotový.")


C:\Users\Kristián\AppData\Local\Temp\ipykernel_21116\1740559156.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest = gdf.geometry.distance(Point(lon, lat)).idxmin()
C:\Users\Kristián\AppData\Local\Temp\ipykernel_21116\1740559156.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest = gdf.geometry.distance(Point(lon, lat)).idxmin()
C:\Users\Kristián\AppData\Local\Temp\ipykernel_21116\1740559156.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest = gdf.geometry.distance(Point(lon, lat)).idxmin()
C:\Users\Kristián\AppData\Local\Temp\ipykernel_21116

✅ Vylepšený OSM enrichment hotový.


C:\Users\Kristián\AppData\Local\Temp\ipykernel_21116\1740559156.py:43: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest = gdf.geometry.distance(Point(lon, lat)).idxmin()


In [35]:
import json
import pandas as pd
from tqdm import tqdm

# Cesta k tvojmu final jsonl suboru
file_path = "synteticke_json_jazdy_enriched_full_osm2.jsonl"

# Príprava na ukladanie výsledkov
results = []

# Načítanie riadkov zo súboru
with open(file_path, 'r') as f:
    for line in tqdm(f, desc="Validating records"):
        record = json.loads(line)
        
        row = {}
        
        # Validácia základných enrichment polí
        row['has_road_risk_index'] = 'road_risk_index' in record
        row['has_incident_base_probability'] = 'incident_base_probability' in record
        row['has_stop_and_go'] = 'stop_and_go_events' in record
        row['has_hard_accel'] = 'hard_acceleration_events' in record
        row['has_hard_brake'] = 'hard_braking_events' in record
        row['has_lateral_risk'] = 'lateral_risk_events' in record
        row['has_traffic_api'] = 'traffic_api' in record
        row['has_path_nodes'] = 'path_nodes' in record
        
        # OSM enrichment polia
        row['has_osm_highway'] = 'osm_highway' in record and record['osm_highway'] is not None
        row['has_osm_maxspeed'] = 'osm_maxspeed' in record and record['osm_maxspeed'] is not None
        row['has_osm_lanes'] = 'osm_lanes' in record and record['osm_lanes'] is not None
        row['has_osm_surface'] = 'osm_surface' in record and record['osm_surface'] is not None
        row['has_osm_junction'] = 'osm_junction' in record and record['osm_junction'] is not None

        results.append(row)

# Prevod na dataframe
results_df = pd.DataFrame(results)

# Výpočet % pokrytia
coverage = results_df.mean() * 100

print("====== VALIDATION REPORT ======")
print(coverage)

# Ak chceš detailný prehľad chýbajúcich záznamov
missing_report = (results_df == False).sum()
print("\n====== MISSING VALUES REPORT ======")
print(missing_report)


Validating records: 900it [00:01, 537.00it/s]

====== VALIDATION REPORT ======
has_road_risk_index              100.000000
has_incident_base_probability    100.000000
has_stop_and_go                  100.000000
has_hard_accel                   100.000000
has_hard_brake                   100.000000
has_lateral_risk                 100.000000
has_traffic_api                  100.000000
has_path_nodes                   100.000000
has_osm_highway                   87.444444
has_osm_maxspeed                  87.444444
has_osm_lanes                     87.444444
has_osm_surface                   87.444444
has_osm_junction                  87.444444
dtype: float64

====== MISSING VALUES REPORT ======
has_road_risk_index                0
has_incident_base_probability      0
has_stop_and_go                    0
has_hard_accel                     0
has_hard_brake                     0
has_lateral_risk                   0
has_traffic_api                    0
has_path_nodes                     0
has_osm_highway                  113
has_osm_max

In [24]:
import json
import random

# --- Pomocné funkcie ---
def classify_severity(incident_type):
    if incident_type in ["sudden_slowdown", "contextual"]:
        return "low"
    if incident_type in ["sharp_turn_high_speed", "aggressive_accel_in_risk_env", "risky_turn_at_junction"]:
        return "medium"
    return "high"

def detect_complex_incidents(records, scenario, driver_type, weather_condition, osm_data=None):
    incidents = {}
    prev_speed = None
    prev_heading = None
    lat_acc_history = []
    heading_history = []
    speed_history = []
    accel_history = []

    road_state = osm_data.get('road_state', '') if osm_data else ''
    raw_lanes = osm_data.get('osm_lanes', 1)
    try:
        osm_lanes = int(raw_lanes)
    except (ValueError, TypeError):
        osm_lanes = 1
    osm_junction = osm_data.get('osm_junction', '') if osm_data else ''

    last_incident_time = -99999
    MIN_TIME_DIFF = 15

    for i, rec in enumerate(records):
        if rec["n"] != "SPEED":
            continue
        try:
            t = rec['t']
            speed = float(rec["v"])
            accel = next((float(r["v"]) for r in records[i:i+5] if r["n"] == "LONGITUDINAL_ACCELERATION"), 0.0)
            lat_acc = next((float(r["v"]) for r in records[i:i+5] if r["n"] == "LATERAL_ACCELERATION"), 0.0)
            heading = next((int(r["v"]) for r in records[i:i+5] if r["n"] == "HEADING"), None)
        except:
            continue

        if t - last_incident_time < MIN_TIME_DIFF:
            continue

        if speed > 60 and accel < -5.0:
            incidents[t] = "hard_brake_high_speed"
            last_incident_time = t
        elif abs(lat_acc) > 3.8 and speed > 50:
            incidents[t] = "sharp_turn_high_speed"
            last_incident_time = t
        elif accel > 4.5 and (scenario == 'mesto' or weather_condition == 'rain'):
            incidents[t] = "aggressive_accel_in_risk_env"
            last_incident_time = t
        elif prev_speed and prev_speed > 80 and speed < 20:
            incidents[t] = "sudden_slowdown"
            last_incident_time = t
        elif prev_heading is not None and heading is not None and abs(heading - prev_heading) > 100:
            incidents[t] = "sudden_direction_change"
            last_incident_time = t
        elif len(lat_acc_history) >= 3 and all(val > 2.8 for val in lat_acc_history[-3:]):
            incidents[t] = "sustained_lateral_instability"
            last_incident_time = t
        elif osm_junction in ['intersection', 'roundabout'] and speed < 15 and abs(lat_acc) > 3.0:
            incidents[t] = "risky_turn_at_junction"
            last_incident_time = t
        elif osm_lanes >= 2 and abs(lat_acc) > 3.2 and prev_heading is not None and heading is not None and abs(heading - prev_heading) > 40:
            incidents[t] = "lane_change_risk"
            last_incident_time = t
        elif road_state == 'icy' and abs(accel) > 4.8 and scenario in ['vidiek', 'diaľnica']:
            incidents[t] = "slippery_surface_maneuver"
            last_incident_time = t
        elif road_state == 'wet' and abs(lat_acc) > 4 and scenario == 'diaľnica':
            incidents[t] = "slippery_surface_maneuver"
            last_incident_time = t
        elif len(speed_history) >= 3 and speed_history[-3] < 5 and accel > 4.5:
            incidents[t] = "sudden_start_collision_risk"
            last_incident_time = t

        lat_acc_history.append(abs(lat_acc))
        speed_history.append(speed)
        accel_history.append(accel)
        prev_heading = heading
        prev_speed = speed

    if driver_type == 'agresívny' and scenario == 'mesto' and weather_condition in ['rain', 'fog']:
        if len(records) > 10:
            t = records[10]['t']
            if t - last_incident_time >= MIN_TIME_DIFF:
                incidents[t] = "aggressive_driver_in_risk_env"

    return incidents

def generate_contextual_incidents(records, scenario, driver_type, weather_condition):
    prob = 0.0007
    if scenario == "mesto":
        prob += 0.004
    if scenario == "vidiek":
        prob += 0.002
    if driver_type == "agresívny":
        prob += 0.006
    if weather_condition in ["rain", "fog", "snow"]:
        prob += 0.008

    timestamps = set()
    max_contextual = 2
    count = 0
    for rec in records:
        if rec["n"] == "SPEED" and count < max_contextual:
            if random.random() < prob:
                timestamps.add(rec["t"])
                count += 1
    return timestamps

# --- Hlavný skript ---
def enrich_incidents(input_path, output_path):
    with open(input_path, "r") as fin, open(output_path, "w") as fout:
        for line in fin:
            drive = json.loads(line)
            records = drive["records"]
            scenario = drive.get("scenario", "mesto")
            driver_type = drive.get("driverType", "normálny")
            weather_condition = drive.get("weather", {}).get("condition", "clear")
            osm_data = {
                'osm_lanes': drive.get('osm_lanes', 1),
                'osm_junction': drive.get('osm_junction', ''),
                'road_state': drive.get('weather', {}).get('road_state', '')
            }

            complex_incidents = detect_complex_incidents(records, scenario, driver_type, weather_condition, osm_data)
            contextual_incidents = generate_contextual_incidents(records, scenario, driver_type, weather_condition) if random.random() < 0.6 else set()

            for rec in records:
                if rec["n"] == "INCIDENT":
                    t = rec["t"]
                    if t in complex_incidents:
                        incident_type = complex_incidents[t]
                        rec["v"] = "1"
                        rec["incident_type"] = incident_type
                        rec["incident_severity"] = classify_severity(incident_type)
                    elif t in contextual_incidents:
                        rec["v"] = "1"
                        rec["incident_type"] = "contextual"
                        rec["incident_severity"] = classify_severity("contextual")
                    else:
                        rec["v"] = "0"

            drive["records"] = records
            fout.write(json.dumps(drive) + "\n")

    print(f"✅ Incident enrichment hotov. Upravený súbor: {output_path}")


In [48]:

enrich_incidents("synteticke_json_jazdy_enriched_full_osm2.jsonl", "synteticke_json_jazdy_enriched_full_osm3.jsonl")


✅ Incident enrichment hotov. Upravený súbor: synteticke_json_jazdy_enriched_full_osm3.jsonl


In [49]:
import json
import pandas as pd
from collections import Counter

# --- Overenie incidentov a ich typov ---
def validate_incident_annotations(input_file, max_records=None):
    incident_counts = []
    type_counter = Counter()
    total_incidents = 0
    total_drives = 0

    with open(input_file, 'r') as f:
        for i, line in enumerate(f):
            if max_records and i >= max_records:
                break
            drive = json.loads(line)
            incident_times = 0
            incident_types = []

            for rec in drive.get("records", []):
                if rec.get("n") == "INCIDENT" and rec.get("v") == "1":
                    incident_times += 1
                    itype = rec.get("incident_type", "unknown")
                    incident_types.append(itype)
                    type_counter[itype] += 1

            total_incidents += incident_times
            total_drives += 1

            incident_counts.append({
                "drive_id": i,
                "scenario": drive.get("scenario"),
                "driver_type": drive.get("driverType"),
                "incident_count": incident_times,
                "incident_types": incident_types
            })

    df = pd.DataFrame(incident_counts)
    summary = {
        "total_drives": total_drives,
        "drives_with_incidents": df[df["incident_count"] > 0].shape[0],
        "total_incidents": total_incidents,
        "average_incidents_per_drive": round(total_incidents / total_drives, 2) if total_drives else 0,
        "incident_type_distribution": dict(type_counter)
    }

    return df, summary

# Príklad použitia:
df_incidents, summary = validate_incident_annotations("synteticke_json_jazdy_enriched_full_osm3.jsonl")
print(summary)
print(df_incidents.head())


{'total_drives': 900, 'drives_with_incidents': 899, 'total_incidents': 23008, 'average_incidents_per_drive': 25.56, 'incident_type_distribution': {'aggressive_accel_in_risk_env': 5996, 'contextual': 716, 'slippery_surface_maneuver': 2291, 'hard_brake_high_speed': 12572, 'sharp_turn_high_speed': 1430, 'sustained_lateral_instability': 2, 'sudden_slowdown': 1}}
   drive_id scenario driver_type  incident_count  \
0         0    mesto  defenzívny               5   
1         1    mesto    normálny              11   
2         2    mesto   agresívny              18   
3         3   vidiek  defenzívny              36   
4         4   vidiek    normálny              20   

                                      incident_types  
0  [aggressive_accel_in_risk_env, aggressive_acce...  
1  [aggressive_accel_in_risk_env, aggressive_acce...  
2  [aggressive_accel_in_risk_env, aggressive_acce...  
3  [slippery_surface_maneuver, hard_brake_high_sp...  
4  [slippery_surface_maneuver, hard_brake_high_sp..

In [50]:
import json
import pandas as pd
from collections import defaultdict

input_path = "synteticke_json_jazdy_enriched_full_osm3.jsonl"

# Zber počtov incidentov podľa driverType
driver_data = defaultdict(lambda: {"incident_count": 0, "drive_count": 0})

with open(input_path, "r") as f:
    for line in f:
        drive = json.loads(line)
        driver = drive.get("driverType", "neznamy")
        incidents = [r for r in drive["records"] if r["n"] == "INCIDENT" and r["v"] == "1"]
        driver_data[driver]["incident_count"] += len(incidents)
        driver_data[driver]["drive_count"] += 1

# Výpočet priemerov
rows = []
for driver, data in driver_data.items():
    avg = data["incident_count"] / data["drive_count"] if data["drive_count"] > 0 else 0
    rows.append({"driverType": driver, "drives": data["drive_count"], "total_incidents": data["incident_count"], "avg_incidents": round(avg, 2)})

df = pd.DataFrame(rows)
print(df.sort_values("avg_incidents", ascending=False))


   driverType  drives  total_incidents  avg_incidents
2   agresívny     300             8743          29.14
1    normálny     300             7570          25.23
0  defenzívny     300             6695          22.32


In [51]:
import json
from collections import Counter
import numpy as np

# Cesta k súboru
input_path = "synteticke_json_jazdy_enriched_full_osm3.jsonl"

# Inicializácia
scenario_counts = Counter()
driver_type_counts = Counter()
weather_condition_counts = Counter()
stop_and_go_counts = []
hard_accel_counts = []
hard_brake_counts = []
lateral_risk_counts = []
risk_indices = []
incident_probs = []
osm_maxspeeds = Counter()

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        drive = json.loads(line)
        scenario_counts[drive.get("scenario", "unknown")] += 1
        driver_type_counts[drive.get("driverType", "unknown")] += 1
        weather_condition_counts[drive.get("weather", {}).get("condition", "unknown")] += 1
        stop_and_go_counts.append(drive.get("stop_and_go_events", 0))
        hard_accel_counts.append(drive.get("hard_acceleration_events", 0))
        hard_brake_counts.append(drive.get("hard_braking_events", 0))
        lateral_risk_counts.append(drive.get("lateral_risk_events", 0))
        risk_indices.append(drive.get("road_risk_index", 0))
        incident_probs.append(drive.get("incident_base_probability", 0))
        osm_maxspeeds[str(drive.get("osm_maxspeed", "unknown"))] += 1

# Výpis prehľadu
print(f"Počet jázd: {len(stop_and_go_counts)}")
print("Scenáre:", dict(scenario_counts))
print("Typy vodičov:", dict(driver_type_counts))
print("Počet podľa stavu počasia:", dict(weather_condition_counts))
print("Priemerné stop-and-go:", round(np.mean(stop_and_go_counts), 2))
print("Priemerné prudké zrýchlenia:", round(np.mean(hard_accel_counts), 2))
print("Priemerné prudké brzdenia:", round(np.mean(hard_brake_counts), 2))
print("Priemerné laterálne riziká:", round(np.mean(lateral_risk_counts), 2))
print("Rozsah rizikového indexu:", (min(risk_indices), max(risk_indices)))
print("Rozsah incident pravdepodobnosti:", (min(incident_probs), max(incident_probs)))
print("Rozsah maxspeed (OSM):", dict(osm_maxspeeds))


Počet jázd: 900
Scenáre: {'mesto': 300, 'vidiek': 300, 'diaľnica': 300}
Typy vodičov: {'defenzívny': 300, 'normálny': 300, 'agresívny': 300}
Počet podľa stavu počasia: {'snow': 84, 'clear': 468, 'rain': 181, 'fog': 167}
Priemerné stop-and-go: 0.0
Priemerné prudké zrýchlenia: 42.65
Priemerné prudké brzdenia: 50.17
Priemerné laterálne riziká: 7.44
Rozsah rizikového indexu: (1, 114)
Rozsah incident pravdepodobnosti: (0.01, 0.03)
Rozsah maxspeed (OSM): {'30': 27, 'unknown': 414, 'nan': 397, '50': 37, '80': 1, '130': 2, '20': 10, '90': 9, '40': 2, '110': 1}
